#Question 1

Write code to perform the following tasks using GraphFrames:

Find Alice's two-hop neighbors' names, regardless of the edge type.

# Question 2

Redo the previous question, but exclude Alice's two-hop neighbors who have an edge back to Alice.

# Question 3

Find all people who follow Charlie.

Hint: Use AND in SQL, or (..) & (..) in DataFrame boolean expressions.

# Question 4

Find all people who are being followed by at least 2 people.

# Question 5

Create a queue of 10 RDDs using this data set and feed it into a Spark Streaming program.  Your Spark Streaming algorithm should maintain a state that keeps track of the longest noun seen so far associated with each distinct adjective. After each RDD, print any 5 adjectives and their associated longest nouns, as well as the longest noun associated with the adjective 'good'. Note that not every line in the data set contains exactly two words, so make sure to clean the data as they are fed into the streaming program.  The skeleton code is provided below:


```python

from pyspark.streaming import StreamingContext

ssc = StreamingContext(sc, 5)
# Provide a checkpointing directory. Required for stateful transformations
ssc.checkpoint("checkpoint")

numPartitions = 8
rdd = sc.textFile('../data/adj_noun_pairs.txt', numPartitions)
rddQueue = rdd.randomSplit([1]*10, 123)
lines = ssc.queueStream(rddQueue)

# FILL IN YOUR CODE

ssc.start()
ssc.awaitTermination(50)
ssc.stop(False)
print("Finished")
```